# Testing

In [1]:
# Prediction margin: the only parameter to set. Recommended: margin in {5, 10, 15, 20} (aka 0.5, 1, 1.5, 2 seconds)
margin = 5

## Import libraries and define utility functions

In [2]:
### NO GPU
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
import pandas as pd
import numpy as np
import sys
import random
import pickle
from time import time
from sklearn.preprocessing import StandardScaler
from math import floor
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [4]:
mean = lambda l: sum(l) / len(l)

def describe(l, n_quantiles=10, list_quantiles=None):
    print(f'Min={min(l):.4}, Max={max(l):.4}, Avg={mean(l):.4}, Tot={len(l)}')
    if list_quantiles is None:
        list_quantiles = [e / n_quantiles for e in range(1, n_quantiles)]
    for q in list_quantiles:
        print(q, ":", round(np.quantile(l, q), 3))
    return min(l), max(l), mean(l), len(l)

#### Set up global variables

In [5]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [6]:
timings = []
timings_net = []

In [7]:
min_time = 0.00001
models_path = "models_" + str(margin) + "/"
results_path = "results_" + str(margin) + "/"
threshold_path = "threshold_" + str(margin) + "/"
seeds = [100]

In [8]:
w5_features_no_diff = [
 'Gz_mean_w5',
 'Ax_mean_w5',
 'Ay_mean_w5',
 'Gz_std_w5',
 'Ax_std_w5',
 'Ay_std_w5',
 'Gz_min_w5',
 'Ax_min_w5',
 'Ay_min_w5',
 'Gz_max_w5',
 'Ax_max_w5',
 'Ay_max_w5'
]

w5_features_diff = [
 'differencing_Gz_mean_w5',
 'differencing_Ax_mean_w5',
 'differencing_Ay_mean_w5',
 'differencing_Gz_std_w5',
 'differencing_Ax_std_w5',
 'differencing_Ay_std_w5',
 'differencing_Gz_min_w5',
 'differencing_Ax_min_w5',
 'differencing_Ay_min_w5',
 'differencing_Gz_max_w5',
 'differencing_Ax_max_w5',
 'differencing_Ay_max_w5',
]

w10_features_no_diff = [
 'Gz_mean_w10',
 'Ax_mean_w10',
 'Ay_mean_w10',
 'Gz_std_w10',
 'Ax_std_w10',
 'Ay_std_w10',
 'Gz_min_w10',
 'Ax_min_w10',
 'Ay_min_w10',
 'Gz_max_w10',
 'Ax_max_w10',
 'Ay_max_w10'
]

w10_features_diff = [
 'differencing_Gz_mean_w10',
 'differencing_Ax_mean_w10',
 'differencing_Ay_mean_w10',
 'differencing_Gz_std_w10',
 'differencing_Ax_std_w10',
 'differencing_Ay_std_w10',
 'differencing_Gz_min_w10',
 'differencing_Ax_min_w10',
 'differencing_Ay_min_w10',
 'differencing_Gz_max_w10',
 'differencing_Ax_max_w10',
 'differencing_Ay_max_w10'
]

w15_features_no_diff = [
 'Gz_mean_w15',
 'Ax_mean_w15',
 'Ay_mean_w15',
 'Gz_std_w15',
 'Ax_std_w15',
 'Ay_std_w15',
 'Gz_min_w15',
 'Ax_min_w15',
 'Ay_min_w15',
 'Gz_max_w15',
 'Ax_max_w15',
 'Ay_max_w15'
]

w15_features_diff = [
 'differencing_Gz_mean_w15',
 'differencing_Ax_mean_w15',
 'differencing_Ay_mean_w15',
 'differencing_Gz_std_w15',
 'differencing_Ax_std_w15',
 'differencing_Ay_std_w15',
 'differencing_Gz_min_w15',
 'differencing_Ax_min_w15',
 'differencing_Ay_min_w15',
 'differencing_Gz_max_w15',
 'differencing_Ax_max_w15',
 'differencing_Ay_max_w15'
]

w20_features_no_diff = [
 'Gz_mean_w20',
 'Ax_mean_w20',
 'Ay_mean_w20',
 'Gz_std_w20',
 'Ax_std_w20',
 'Ay_std_w20',
 'Gz_min_w20',
 'Ax_min_w20',
 'Ay_min_w20',
 'Gz_max_w20',
 'Ax_max_w20',
 'Ay_max_w20'
]

w20_features_diff = [
 'differencing_Gz_mean_w20',
 'differencing_Ax_mean_w20',
 'differencing_Ay_mean_w20',
 'differencing_Gz_std_w20',
 'differencing_Ax_std_w20',
 'differencing_Ay_std_w20',
 'differencing_Gz_min_w20',
 'differencing_Ax_min_w20',
 'differencing_Ay_min_w20',
 'differencing_Gz_max_w20',
 'differencing_Ax_max_w20',
 'differencing_Ay_max_w20',
]

features = {
    "all_features": w5_features_no_diff + w10_features_no_diff + w15_features_no_diff + w20_features_no_diff + w5_features_diff + w10_features_diff + w15_features_diff + w20_features_diff + ['label'], 
    "w5_features": w5_features_no_diff + w5_features_diff + ['label'], 
    "w10_features": w10_features_no_diff + w10_features_diff + ['label'], 
    "w15_features": w15_features_no_diff + w15_features_diff + ['label'], 
    "w20_features": w20_features_no_diff + w20_features_diff + ['label'], 
    "no_diff_features": w5_features_no_diff + w10_features_no_diff + w15_features_no_diff + w20_features_no_diff + ['label'], 
    "diff_features": w5_features_diff + w10_features_diff + w15_features_diff + w20_features_diff + ['label']
}

#### Set up global variables

In [9]:
f_headers = r'data/headers.txt'
f_acc = r'data/test-raw/dataset_acc.csv'
f_pos = r'data/test-raw/dataset_pos.csv'
f_labels = r'data/test-raw/labels.csv'

#### Load utilities 

In [10]:
with open('data/utils/scaler_' + str(margin) + '.bin', 'rb') as handle:
    scaler = pickle.load(handle)
with open('data/utils/fixed_orient_' + str(margin) + '.bin', 'rb') as handle:
    fixed_orient = pickle.load(handle)
with open('data/utils/fixed_pos_' + str(margin) + '.bin', 'rb') as handle:
    fixed_pos = pickle.load(handle)
with open('data/utils/differencing_dict_' + str(margin) + '.bin', 'rb') as handle:
    differencing_dict = pickle.load(handle) 

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Raw data import

In [11]:
with open(f_headers) as f_headers:
    col_acc = f_headers.readline()[:-1].split(",")
    col_pos = f_headers.readline()[:-1].split(",")
    col_lab = f_headers.readline().split(",")

In [12]:
acc_df = pd.read_csv(f_acc, names=col_acc)
pos_df = pd.read_csv(f_pos, names=col_pos)
lab_df = pd.read_csv(f_labels, names=col_lab)

#### Raw data inspection

In [13]:
print(f'The dataset contains {len(acc_df)} samples')
acc_df.head()

The dataset contains 49979 samples


,time_server,time_client,Gz,Ax,Ay
0,1651503005.5170,1651176129.8823,0.0687,-2.0750,-0.1953
1,1651503005.6167,1651176129.9824,0.0687,-2.4170,-0.6104
2,1651503005.7167,1651176130.0825,0.0687,-1.5870,-0.0244
3,1651503005.8166,1651176130.1828,0.0687,-2.0260,-0.9766
4,1651503005.9165,1651176130.2830,0.0763,-2.0020,-0.6104


In [14]:
print(f'The dataset contains {len(pos_df)} samples')
pos_df.head()

The dataset contains 25338 samples


,time_server,POSx,POSy,orient
0,1651505818.5333,31.7187,29.1164,52.2161
1,1651505818.6358,31.7321,29.1264,54.4788
2,1651505818.7355,31.7467,29.1363,56.8535
3,1651505818.8354,31.7617,29.1457,59.1696
4,1651505818.9354,31.7762,29.1632,61.2057


In [15]:
print(f'The dataset contains {len(lab_df)} samples')
lab_df.head()

The dataset contains 26 samples


,time_server,label
0,1651505990.9049,s
1,1651506204.2696,f
2,1651506232.7222,s
3,1651506368.3697,f
4,1651506397.5757,s


## Data merge

In [16]:
def find_nearest(array, value, return_index=True):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    if return_index:
        return array[idx], idx
    else:
        return array[idx]

def merge_data(acc, pos, lab):
    
    delta = 0.05
    merged_data = []
    disalignments = []
    
    for i_lab in range(len(lab) - 1):
        if lab.loc[i_lab, "label"] == "s":
            start = lab.loc[i_lab, "time_server"]
            end = lab.loc[i_lab + 1, "time_server"]
            curr_acc = acc[(acc["time_server"] > start - delta) & (acc["time_server"] < end + delta)]
            curr_pos = pos[(pos["time_server"] > start - delta) & (pos["time_server"] < end + delta)]
            
            for i_acc in range(len(curr_acc)):
                curr_data_acc = curr_acc.iloc[i_acc]
                curr_time = curr_data_acc["time_server"]
                curr_data_acc = curr_data_acc.tolist()
                nearest, i_nearest = find_nearest(curr_pos["time_server"].to_numpy(), curr_time)
                disalignments.append(np.abs(curr_time - nearest))
                curr_data_pos = curr_pos.iloc[i_nearest].tolist()
                merged_data.append(curr_data_acc + curr_data_pos + [len(curr_acc) - 1 - i_acc])
                
            print(f'The {int(i_lab / 2 + 1)}° lap lasts for approx. {len(curr_acc) / 10} seconds')
        
    return merged_data, disalignments

In [17]:
merged_data, time_disalignments = merge_data(acc_df, pos_df, lab_df)

The 1° lap lasts for approx. 210.1 seconds
The 2° lap lasts for approx. 133.3 seconds
The 3° lap lasts for approx. 106.3 seconds
The 4° lap lasts for approx. 135.8 seconds
The 5° lap lasts for approx. 95.7 seconds
The 6° lap lasts for approx. 99.7 seconds
The 7° lap lasts for approx. 96.8 seconds
The 8° lap lasts for approx. 96.7 seconds
The 9° lap lasts for approx. 95.4 seconds
The 10° lap lasts for approx. 135.3 seconds
The 11° lap lasts for approx. 93.4 seconds
The 12° lap lasts for approx. 94.7 seconds
The 13° lap lasts for approx. 24.3 seconds


In [18]:
raw_df = pd.DataFrame(merged_data, columns=col_acc+["time_server_pos"]+col_pos[1:]+["label"])
raw_df

,time_server,time_client,Gz,Ax,Ay,time_server_pos,POSx,POSy,orient,label
0,1651505990.8958,1651179115.2670,-0.4809,0.8057,-1.8550,1651505990.9168,31.4612,20.5103,-4.5177,2100
1,1651505990.9959,1651179115.3674,-0.4427,-3.9060,1.9530,1651505991.0180,31.4529,20.6053,-5.4122,2099
2,1651505991.0459,1651179115.4177,-0.4809,-2.2460,-1.8310,1651505991.0180,31.4529,20.6053,-5.4122,2098
3,1651505991.1959,1651179115.5682,-0.3511,-1.3430,-5.4930,1651505991.2230,31.4423,20.7088,-6.1759,2097
4,1651505991.2958,1651179115.6688,-0.2901,-0.0732,-1.8310,1651505991.3341,31.4336,20.7628,-6.1412,2096
...,...,...,...,...,...,...,...,...,...,...
14170,1651508387.1363,1651181511.5192,-0.0611,-11.8200,-5.1510,1651508387.0684,31.1923,26.9123,-155.9550,4
14171,1651508387.2367,1651181511.6195,0.1145,-3.5640,-5.9330,1651508387.2108,31.1323,26.7749,-156.3772,3
14172,1651508387.3365,1651181511.7200,-0.0229,-29.0300,-6.2740,1651508387.3132,31.1065,26.7159,-156.3608,2
14173,1651508387.4366,1651181511.8204,-0.4275,-31.3700,-1.8310,1651508387.4155,31.0975,26.6952,-156.4760,1


In [19]:
n_sample_test_set = len(raw_df)

In [20]:
min_td, max_td, mean_td, len_td = describe(time_disalignments, list_quantiles=[.5, .9, .99, .999])

Min=3.099e-06, Max=2.52, Avg=0.04078, Tot=14175
0.5 : 0.037
0.9 : 0.049
0.99 : 0.148
0.999 : 1.815


In [21]:
print(f'On average, we have wait {mean_td:.4f} to merge acc e pos information')

On average, we have wait 0.0408 to merge acc e pos information


In [22]:
timings.append({"name": "merge_time", "time": mean_td})

## Data processing

In [23]:
useful_columns_raw = ["Gz", "Ax", "Ay", 'POSx', 'POSy', 'orient', "label"]

In [24]:
start = time()
raw_df_reduced = raw_df[useful_columns_raw]
end = (time() - start) / n_sample_test_set
print(f"Executed in {end} seconds")
if end < min_time:
    print("EXECUTION TIME NEGLIGIBLE")
else:
    timings.append({"name": "filter0_time", "time": end})

Executed in 4.7313683457685734e-08 seconds
EXECUTION TIME NEGLIGIBLE


In [25]:
# THIS OPERATION IS NOT PERFORMED AT INFERENCE TIME
df_split = []
fault_indexes = raw_df_reduced.index[raw_df_reduced["label"] == 0].tolist() # list of indexes representing faults
        
previous = 0
for fi in fault_indexes:
    df_split.append(raw_df_reduced.iloc[previous:fi+1, :])
    previous = fi + 1

In [26]:
# THIS OPERATION IS NOT PERFORMED AT INFERENCE TIME
print(f'There are {len(df_split)} faults, hence {len(df_split)} datasets.')
for i, df_tmp in enumerate(df_split):
    print(i, df_tmp.shape)

There are 13 faults, hence 13 datasets.
0 (2101, 7)
1 (1333, 7)
2 (1063, 7)
3 (1358, 7)
4 (957, 7)
5 (997, 7)
6 (968, 7)
7 (967, 7)
8 (954, 7)
9 (1353, 7)
10 (934, 7)
11 (947, 7)
12 (243, 7)


In [27]:
# reduce size
df_split[0] = df_split[0][:40]  # 20 is max lag
df_split = df_split[:1]
df_split[0]

,Gz,Ax,Ay,POSx,POSy,orient,label
0,-0.4809,0.8057,-1.8550,31.4612,20.5103,-4.5177,2100
1,-0.4427,-3.9060,1.9530,31.4529,20.6053,-5.4122,2099
2,-0.4809,-2.2460,-1.8310,31.4529,20.6053,-5.4122,2098
3,-0.3511,-1.3430,-5.4930,31.4423,20.7088,-6.1759,2097
4,-0.2901,-0.0732,-1.8310,31.4336,20.7628,-6.1412,2096
5,-0.1298,-1.9290,-1.8550,31.4275,20.8186,-6.3637,2095
6,-0.0534,0.2930,-0.9521,31.4209,20.8773,-6.4775,2094
7,-0.0229,3.5400,-2.1730,31.4140,20.9380,-6.5494,2093
8,0.0229,-0.8789,2.1730,31.4140,20.9380,-6.5494,2092
9,0.0229,2.5630,0.0000,31.3972,21.0727,-6.8706,2091


## Features creation

In [28]:
w_lens = [5, 10, 15, 20]

In [29]:
new_dfs = []
lag_features = []
first = True

for temps in df_split:
    curr_w_len_data = []
    curr_n_sample_test_set = len(temps)
    for w_len in w_lens:
        
        if first:
            start = time()
            
        means = temps.rolling(w_len).mean()

        if first:
            end = (time() - start) / curr_n_sample_test_set
            print(f"Executed in {end} seconds")
            if end < min_time:
                print("EXECUTION TIME NEGLIGIBLE")
            else:
                timings.append({"name": f"lag_mean_{w_len}", "time": end})
        
        cols = [t + "_mean_w" + str(w_len) for t in temps.columns]
        means.columns = cols 
        curr_w_len_data.append(means)
        if first:
            lag_features.append(cols)
            
        if first:
            start = time()
        
        stds = temps.rolling(w_len).std()
        
        if first:
            end = (time() - start) / curr_n_sample_test_set
            print(f"Executed in {end} seconds")
            if end < min_time:
                print("EXECUTION TIME NEGLIGIBLE")
            else:
                timings.append({"name": f"lag_std_{w_len}", "time": end})
        
        cols = [t + "_std_w" + str(w_len) for t in temps.columns]
        stds.columns = cols
        curr_w_len_data.append(stds)
        if first:
            lag_features.append(cols)
            
        if first:
            start = time()

        mins = temps.rolling(w_len).min()
        
        if first:
            end = (time() - start) / curr_n_sample_test_set
            print(f"Executed in {end} seconds")
            if end < min_time:
                print("EXECUTION TIME NEGLIGIBLE")
            else:
                timings.append({"name": f"lag_min_{w_len}", "time": end})
        
        cols = [t + "_min_w" + str(w_len) for t in temps.columns]
        mins.columns = cols
        curr_w_len_data.append(mins)
        if first:
            lag_features.append(cols)
            
        if first:
            start = time()

        maxs = temps.rolling(w_len).max()
        
        if first:
            end = (time() - start) / curr_n_sample_test_set
            print(f"Executed in {end} seconds")
            if end < min_time:
                print("EXECUTION TIME NEGLIGIBLE")
            else:
                timings.append({"name": f"lag_max_{w_len}", "time": end})
        
        cols = [t + "_max_w" + str(w_len) for t in temps.columns]
        maxs.columns = cols
        curr_w_len_data.append(maxs)
        if first:
            lag_features.append(cols)

    if first:
        start = time()        
    
    temps_diff = temps - temps.shift(1)
    
    if first:
        end = (time() - start) / curr_n_sample_test_set
        print(f"Executed in {end} seconds")
        if end < min_time:
            print("EXECUTION TIME NEGLIGIBLE")
        else:
            timings.append({"name": f"lag_diff_{w_len}", "time": end})
    
    temps_diff.columns = [t + "_diff" for t in temps.columns]

    df_with_nan = pd.concat([temps, temps_diff] + curr_w_len_data, axis=1)
    df_curr = df_with_nan.dropna()

    new_dfs.append(df_curr)
    
    first = False

df_new_features = new_dfs[0]
for to_concat in new_dfs[1:]:
    df_new_features = pd.concat([df_new_features, to_concat])
    
lag_features = [e for nested_lag_features in lag_features for e in nested_lag_features 
                if not e.startswith("POS") and not e.startswith("orient") and not e.startswith("label")]

Executed in 1.3244152069091796e-05 seconds
Executed in 1.4382600784301758e-05 seconds
Executed in 8.481740951538087e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 8.803606033325195e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 1.4579296112060547e-05 seconds
Executed in 1.246333122253418e-05 seconds
Executed in 9.781122207641601e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 8.100271224975586e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 1.3870000839233398e-05 seconds
Executed in 1.2326240539550782e-05 seconds
Executed in 9.506940841674805e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 1.055598258972168e-05 seconds
Executed in 1.0412931442260742e-05 seconds
Executed in 1.3327598571777343e-05 seconds
Executed in 9.572505950927734e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 1.0645389556884765e-05 seconds
Executed in 9.936094284057617e-06 seconds
EXECUTION TIME NEGLIGIBLE


In [30]:
point_features = ['Gz', 'Ax', 'Ay', 'Gz_diff', 
                  'Ax_diff', 'Ay_diff']
differencing_features = point_features + lag_features
new_features = differencing_features + ['POSx', 'POSy', 'orient', 'label']
df_new_features = df_new_features[new_features]
df_new_features = df_new_features.reset_index(drop=True)

In [31]:
processed_df_n_samples = len(df_new_features)

In [32]:
df_new_features.head()

,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,Gz_min_w20,Ax_min_w20,Ay_min_w20,Gz_max_w20,Ax_max_w20,Ay_max_w20,POSx,POSy,orient,label
0,0.5573,3.5640,-0.2930,-0.0381,6.1760,0.4150,0.5786,0.4101,2.4854,0.0198,...,-0.4809,-3.9060,-5.4930,0.6031,4.1990,6.8850,31.3119,21.9529,-3.7765,2081
1,0.4198,-0.6836,1.5140,-0.1375,-4.2476,1.8070,0.5496,0.6982,1.4112,0.0748,...,-0.4809,-3.9060,-5.4930,0.6031,4.1990,6.8850,31.3032,22.0414,-4.0979,2080
2,0.3588,7.3970,3.7840,-0.0610,8.0806,2.2700,0.5069,2.3729,1.7334,0.1108,...,-0.4809,-3.3940,-5.4930,0.6031,7.3970,6.8850,31.2962,22.1430,-3.8150,2079
3,0.5191,3.0270,1.6360,0.1603,-4.3700,-2.1480,0.4901,2.1385,1.1866,0.0983,...,-0.3511,-3.3940,-5.4930,0.6031,7.3970,6.8850,31.2894,22.2473,-3.5266,2078
4,0.6031,3.1010,4.1750,0.0840,0.0740,2.5390,0.4916,3.2811,2.1632,0.1004,...,-0.2901,-3.3940,-4.3460,0.6031,7.3970,6.8850,31.2765,22.3468,-3.2053,2077


In [33]:
df_new_features.describe()

,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,Gz_min_w20,Ax_min_w20,Ay_min_w20,Gz_max_w20,Ax_max_w20,Ay_max_w20,POSx,POSy,orient,label
count,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,...,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000
mean,0.5165,0.3183,1.0197,-0.0105,0.2174,-0.0779,0.5360,0.2987,0.8671,0.0881,...,0.0607,-7.2147,-9.1237,0.7136,13.3576,7.3401,31.2538,23.0327,-1.1752,2071.0000
std,0.1284,5.3193,4.9735,0.0976,7.6971,5.9218,0.0847,2.6388,2.4142,0.0367,...,0.3180,2.5860,2.2230,0.0633,4.2204,1.2831,0.0280,0.6975,1.7778,6.2048
min,0.3435,-9.5210,-10.3300,-0.1679,-22.6020,-14.5050,0.3939,-4.0186,-3.4228,0.0198,...,-0.4809,-9.5210,-10.3300,0.6031,4.1990,6.1040,31.2216,21.9529,-4.0979,2061.0000
25%,0.4046,-2.4170,-1.9530,-0.0992,-4.2476,-3.4670,0.4901,-2.1874,-0.7178,0.0616,...,-0.1298,-9.5210,-10.3300,0.7481,15.6200,6.8850,31.2312,22.4588,-2.7459,2066.0000
50%,0.5115,-0.6836,1.6360,0.0153,0.2930,0.4150,0.5496,0.4101,0.2834,0.0848,...,0.1450,-6.9820,-10.3300,0.7481,15.6200,6.8850,31.2429,23.0479,-0.9338,2071.0000
75%,0.6183,3.1010,4.1750,0.0610,5.3220,3.6420,0.6046,2.1385,1.7920,0.1108,...,0.3588,-3.9060,-10.3300,0.7481,15.6200,6.8850,31.2707,23.5184,0.3247,2076.0000
max,0.7481,15.6200,10.3800,0.1603,12.5190,12.1850,0.6657,5.6923,6.3582,0.1537,...,0.3588,-3.3940,-4.3460,0.7481,15.6200,10.3800,31.3119,24.2202,1.3097,2081.0000


In [34]:
df_new_features.columns.tolist()

['Gz',
 'Ax',
 'Ay',
 'Gz_diff',
 'Ax_diff',
 'Ay_diff',
 'Gz_mean_w5',
 'Ax_mean_w5',
 'Ay_mean_w5',
 'Gz_std_w5',
 'Ax_std_w5',
 'Ay_std_w5',
 'Gz_min_w5',
 'Ax_min_w5',
 'Ay_min_w5',
 'Gz_max_w5',
 'Ax_max_w5',
 'Ay_max_w5',
 'Gz_mean_w10',
 'Ax_mean_w10',
 'Ay_mean_w10',
 'Gz_std_w10',
 'Ax_std_w10',
 'Ay_std_w10',
 'Gz_min_w10',
 'Ax_min_w10',
 'Ay_min_w10',
 'Gz_max_w10',
 'Ax_max_w10',
 'Ay_max_w10',
 'Gz_mean_w15',
 'Ax_mean_w15',
 'Ay_mean_w15',
 'Gz_std_w15',
 'Ax_std_w15',
 'Ay_std_w15',
 'Gz_min_w15',
 'Ax_min_w15',
 'Ay_min_w15',
 'Gz_max_w15',
 'Ax_max_w15',
 'Ay_max_w15',
 'Gz_mean_w20',
 'Ax_mean_w20',
 'Ay_mean_w20',
 'Gz_std_w20',
 'Ax_std_w20',
 'Ay_std_w20',
 'Gz_min_w20',
 'Ax_min_w20',
 'Ay_min_w20',
 'Gz_max_w20',
 'Ax_max_w20',
 'Ay_max_w20',
 'POSx',
 'POSy',
 'orient',
 'label']

## Handling seasonality with differencing over position and orientation

In [35]:
# THIS OPERATION IS NOT PERFORMED AT INFERENCE TIME
df_new_features["orient_discr"] = pd.Series([1 if (orient > fixed_orient[0][0] and orient < fixed_orient[0][1]) 
                                             else -1 if (orient < fixed_orient[1][0] or orient > fixed_orient[1][1]) 
                                             else 0 for orient in df_new_features["orient"]])
df_new_features["POSy_discr"] = pd.Series([find_nearest(fixed_pos, curr_pos, return_index=False) 
                                           for curr_pos in df_new_features["POSy"]])

In [36]:
df_new_features.head()

,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,Ay_min_w20,Gz_max_w20,Ax_max_w20,Ay_max_w20,POSx,POSy,orient,label,orient_discr,POSy_discr
0,0.5573,3.5640,-0.2930,-0.0381,6.1760,0.4150,0.5786,0.4101,2.4854,0.0198,...,-5.4930,0.6031,4.1990,6.8850,31.3119,21.9529,-3.7765,2081,1,22.0000
1,0.4198,-0.6836,1.5140,-0.1375,-4.2476,1.8070,0.5496,0.6982,1.4112,0.0748,...,-5.4930,0.6031,4.1990,6.8850,31.3032,22.0414,-4.0979,2080,1,22.0000
2,0.3588,7.3970,3.7840,-0.0610,8.0806,2.2700,0.5069,2.3729,1.7334,0.1108,...,-5.4930,0.6031,7.3970,6.8850,31.2962,22.1430,-3.8150,2079,1,22.1000
3,0.5191,3.0270,1.6360,0.1603,-4.3700,-2.1480,0.4901,2.1385,1.1866,0.0983,...,-5.4930,0.6031,7.3970,6.8850,31.2894,22.2473,-3.5266,2078,1,22.2000
4,0.6031,3.1010,4.1750,0.0840,0.0740,2.5390,0.4916,3.2811,2.1632,0.1004,...,-4.3460,0.6031,7.3970,6.8850,31.2765,22.3468,-3.2053,2077,1,22.3000


In [37]:
differencing_list = {feature: [] for feature in differencing_features}
all_keys = list(differencing_dict["Gz"].keys())

first = True

for _, row in df_new_features.iterrows():
    
    for feature in differencing_features:
        
        if first:
            start = time()
        
        try:
            differencing_list[feature].append(row[feature] - differencing_dict[feature][(row["orient_discr"], row["POSy_discr"])])
        # KeyError: some of the positions in the test set are not available in the training set
        except KeyError:
            same_orient = [e for e in all_keys if e[0] == row["orient_discr"]]
            closest_so = 9999
            for so in same_orient:
                if abs(so[1] - row["POSy_discr"]) < closest_so:
                    best_so = so
            differencing_list[feature].append(row[feature] - differencing_dict[feature][best_so])
    
        if first:
            end = (time() - start)  # just for one row, no division by "processed_df_n_samples" needed
            print(f"Executed in {end} seconds")
            if end < min_time:
                print("EXECUTION TIME NEGLIGIBLE")
            else:
                timings.append({"name": "differencing_" + feature, "time": end})
            
    first = False
                       
for feature in differencing_features:
    df_new_features["differencing_" + feature] = pd.Series(differencing_list[feature])

df_new_features.head()

Executed in 1.71661376953125e-05 seconds
Executed in 8.344650268554688e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.198883056640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.67572021484375e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 9.5367431640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.67572021484375e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.198883056640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.4373016357421875e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 5.9604644775390625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.4373016357421875e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.198883056640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.67572021484375e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.67572021484375e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.198883056640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed in 6.198883056640625e-06 seconds
EXECUTION TIME NEGLIGIBLE
Executed i

,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,differencing_Ay_mean_w20,differencing_Gz_std_w20,differencing_Ax_std_w20,differencing_Ay_std_w20,differencing_Gz_min_w20,differencing_Ax_min_w20,differencing_Ay_min_w20,differencing_Gz_max_w20,differencing_Ax_max_w20,differencing_Ay_max_w20
0,0.5573,3.5640,-0.2930,-0.0381,6.1760,0.4150,0.5786,0.4101,2.4854,0.0198,...,12.4785,0.0206,-0.3517,-1.1179,0.0027,-14.4887,15.0739,-0.4015,-16.8227,11.4140
1,0.4198,-0.6836,1.5140,-0.1375,-4.2476,1.8070,0.5496,0.6982,1.4112,0.0748,...,12.6469,-0.0001,-0.3609,-1.1238,0.0027,-14.4887,15.0739,-0.4015,-16.8227,11.4140
2,0.3588,7.3970,3.7840,-0.0610,8.0806,2.2700,0.5069,2.3729,1.7334,0.1108,...,12.7689,-0.0219,0.0468,-1.1842,-0.0015,-13.9005,15.4786,-0.3870,-13.5096,11.5254
3,0.5191,3.0270,1.6360,0.1603,-4.3700,-2.1480,0.4901,2.1385,1.1866,0.0983,...,12.9595,-0.0419,-0.0521,-1.4446,0.1084,-13.7198,15.9500,-0.3729,-13.7971,10.6884
4,0.6031,3.1010,4.1750,0.0840,0.0740,2.5390,0.4916,3.2811,2.1632,0.1004,...,13.4665,-0.0413,-0.0282,-1.8361,0.0881,-13.7711,17.4649,-0.3345,-13.6622,10.5459


In [38]:
df_new_features.columns

Index(['Gz', 'Ax', 'Ay', 'Gz_diff', 'Ax_diff', 'Ay_diff', 'Gz_mean_w5',
       'Ax_mean_w5', 'Ay_mean_w5', 'Gz_std_w5',
       ...
       'differencing_Ay_mean_w20', 'differencing_Gz_std_w20',
       'differencing_Ax_std_w20', 'differencing_Ay_std_w20',
       'differencing_Gz_min_w20', 'differencing_Ax_min_w20',
       'differencing_Ay_min_w20', 'differencing_Gz_max_w20',
       'differencing_Ax_max_w20', 'differencing_Ay_max_w20'],
      dtype='object', length=114)

In [39]:
timings

[{'name': 'merge_time', 'time': 0.04078222986251589},
 {'name': 'lag_mean_5', 'time': 1.3244152069091796e-05},
 {'name': 'lag_std_5', 'time': 1.4382600784301758e-05},
 {'name': 'lag_mean_10', 'time': 1.4579296112060547e-05},
 {'name': 'lag_std_10', 'time': 1.246333122253418e-05},
 {'name': 'lag_mean_15', 'time': 1.3870000839233398e-05},
 {'name': 'lag_std_15', 'time': 1.2326240539550782e-05},
 {'name': 'lag_max_15', 'time': 1.055598258972168e-05},
 {'name': 'lag_mean_20', 'time': 1.0412931442260742e-05},
 {'name': 'lag_std_20', 'time': 1.3327598571777343e-05},
 {'name': 'lag_max_20', 'time': 1.0645389556884765e-05},
 {'name': 'differencing_Gz', 'time': 1.71661376953125e-05}]

## Features scaling

In [40]:
start = time()
df = pd.DataFrame(scaler.transform(df_new_features), 
                  columns=list(df_new_features.columns))
df = df.drop(columns=["label", 'orient_discr','POSy_discr'])
df["orient_discr"] = df_new_features["orient_discr"]
df["POSy_discr"] = df_new_features["POSy_discr"]
df["label"] = df_new_features["label"]
end = (time() - start)  / processed_df_n_samples
print(f"Executed in {end} seconds")
if end < min_time:
    print("EXECUTION TIME NEGLIGIBLE")
else:
    timings.append({"name": "scaler", "time": end})
df

Executed in 0.00014642306736537388 seconds


,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,differencing_Ay_std_w20,differencing_Gz_min_w20,differencing_Ax_min_w20,differencing_Ay_min_w20,differencing_Gz_max_w20,differencing_Ax_max_w20,differencing_Ay_max_w20,orient_discr,POSy_discr,label
0,-0.1934,-2.2683,2.7542,-0.0889,1.4190,0.0704,-0.1857,-4.3607,6.1935,-0.8680,...,-1.2207,0.0025,-4.7709,4.7837,-0.3088,-6.5228,3.6788,1,22.0000,2081
1,-0.2664,-3.2769,3.1463,-0.3202,-0.9684,0.3066,-0.2014,-4.2621,5.7634,-0.6847,...,-1.2272,0.0025,-4.7709,4.7837,-0.3088,-6.5228,3.6788,1,22.0000,2080
2,-0.2987,-1.3582,3.6389,-0.1422,1.8552,0.3852,-0.2246,-3.6892,5.8924,-0.5648,...,-1.2932,-0.0004,-4.5762,4.9121,-0.2976,-5.2381,3.7147,1,22.1000,2079
3,-0.2137,-2.3959,3.1728,0.3729,-0.9964,-0.3645,-0.2337,-3.7694,5.6735,-0.6066,...,-1.5775,0.0772,-4.5164,5.0616,-0.2867,-5.3495,3.4451,1,22.2000,2078
4,-0.1691,-2.3783,3.7237,0.1953,0.0214,0.4309,-0.2328,-3.3785,6.0645,-0.5996,...,-2.0051,0.0629,-4.5334,5.5422,-0.2572,-5.2972,3.3991,1,22.3000,2077
5,-0.0922,0.5943,0.5763,0.3372,2.8717,-2.4612,-0.2122,-2.5536,5.2608,-0.4221,...,-1.3259,0.0314,-4.3520,3.8296,-0.1191,-2.2468,3.2469,1,22.5000,2076
6,-0.1448,-4.7724,3.2203,-0.2311,-5.1721,2.0676,-0.1874,-2.9846,5.2881,-0.4454,...,-1.3909,0.0104,-5.5000,3.8160,-0.0912,-2.3871,3.1648,1,22.6000,2075
7,-0.1125,-3.5087,2.3940,0.1418,1.2234,-0.6461,-0.1494,-3.6042,4.8288,-0.6339,...,-1.7027,-0.0362,-5.5292,3.9714,-0.0940,-2.2947,2.8620,1,22.7000,2074
8,-0.1611,-3.4392,3.3793,-0.2134,0.0716,0.7706,-0.1387,-3.9049,4.9050,-0.7286,...,-1.8259,-0.0754,-5.4802,3.9747,-0.0304,-2.3503,2.7803,1,22.8000,2073
9,-0.2299,-2.2277,4.1423,-0.3023,1.1730,0.5966,-0.1511,-3.8615,5.0595,-0.6007,...,-1.7329,-0.0876,-5.4141,4.0872,-0.0407,-2.2518,2.7499,1,22.9000,2072


In [41]:
df.describe()

,Gz,Ax,Ay,Gz_diff,Ax_diff,Ay_diff,Gz_mean_w5,Ax_mean_w5,Ay_mean_w5,Gz_std_w5,...,differencing_Ay_std_w20,differencing_Gz_min_w20,differencing_Ax_min_w20,differencing_Ay_min_w20,differencing_Gz_max_w20,differencing_Ax_max_w20,differencing_Ay_max_w20,orient_discr,POSy_discr,label
count,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,...,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000,21.0000
mean,-0.2151,-3.0390,3.0390,-0.0247,0.0543,-0.0132,-0.2088,-4.3987,5.5456,-0.6405,...,-1.6830,0.0899,-5.3669,4.4810,-0.1470,-3.1544,3.0414,1.0000,23.0333,2071.0000
std,0.0681,1.2630,1.0792,0.2271,1.7629,1.0048,0.0458,0.9027,0.9665,0.1221,...,0.4004,0.1176,0.5993,0.4331,0.1264,1.5385,0.5986,0.0000,0.7066,6.2048
min,-0.3069,-5.3753,0.5763,-0.3909,-5.1721,-2.4612,-0.2857,-5.8757,3.8282,-0.8680,...,-2.4089,-0.0876,-6.1709,3.8160,-0.3737,-6.5228,2.1933,1.0000,22.0000,2061.0000
25%,-0.2744,-3.6885,2.3940,-0.2311,-0.9684,-0.5883,-0.2337,-5.2492,4.9111,-0.7286,...,-1.9654,0.0025,-5.8035,4.0960,-0.2867,-2.4534,2.4937,1.0000,22.5000,2066.0000
50%,-0.2177,-3.2769,3.1728,0.0354,0.0716,0.0704,-0.2014,-4.3607,5.3119,-0.6516,...,-1.7329,0.0629,-5.4802,4.5346,-0.0912,-2.3676,2.8620,1.0000,23.0000,2071.0000
75%,-0.1611,-2.3783,3.7237,0.1418,1.2234,0.6180,-0.1717,-3.7694,5.9159,-0.5648,...,-1.3259,0.2011,-4.7709,4.7837,-0.0466,-2.2947,3.6231,1.0000,23.5000,2076.0000
max,-0.0922,0.5943,5.0701,0.3729,2.8717,2.0676,-0.1387,-2.5536,7.7439,-0.4221,...,-0.9592,0.3316,-4.3520,5.5422,-0.0277,-2.2365,4.0100,1.0000,24.2000,2081.0000


# Machine learning timings

### Note: only timings of models' inference are considered, since only them occur in the online phase

## Dataset preprocessing for machine learning models

In this section, RUL labels are converted to binary labels (`0/1`, namely `not_fault/fault`) in order to perform classification instead of regression.

For the `AutoEncoder` model, the dataset is partitioned such that the training set does not contain faults or samples which anticipate a fault. In other words, each sample must be compliant with the `good_samples_thr` threshold.

We basically need an entire section of dataset where faults are not present.

In [42]:
def build_dataset_for_ml_model(df, training_columns, split_size=0.75, as_list=False, ae=False):
    ### START code slightly changed 
    dfs = []
    df_main = df[training_columns]
    good_samples_thr = margin * 2
    
    dfs.append(df_main)
    ### END code slightly changed 
    
    rnd_list = list(range(len(dfs)))
    
    # If split_size is 1, there will be no val/test set
    train_size = floor(len(dfs) * split_size)
    train_index = rnd_list[:train_size]
    test_index = rnd_list[train_size:]
    train_rul = []
    test_rul = []
    
    if not as_list:
        first = True
        for ti in train_index:
            if not ae:
                to_concat = dfs[ti].copy()
            else:
                to_concat = dfs[ti][dfs[ti]["label"] >= good_samples_thr].copy()
            if first:
                training_set = to_concat
                first = False
            else:
                training_set = pd.concat([training_set, to_concat])

        first = True
        for ti in test_index:
            to_concat = dfs[ti].copy()
            if first:
                test_set = to_concat
                first = False
            else:
                test_set = pd.concat([test_set, to_concat])
        
        train_rul = training_set['label'].tolist()
        if split_size < 1:
            test_rul = test_set['label'].tolist()
        
        training_set['label'] = (training_set['label'] >= margin).map({True: 1, False: 0})
        if split_size < 1:
            test_set['label'] = (test_set['label'] >= margin).map({True: 1, False: 0})

        training_set = training_set.to_numpy()
        if split_size < 1:
            test_set = test_set.to_numpy()
        
    else:
        first = True
        for ti in train_index:
            if not ae:
                to_concat = dfs[ti].copy()
            else:
                to_concat = dfs[ti][dfs[ti]["label"] >= good_samples_thr].copy()
            if first:
                training_set = [to_concat]
                first = False
            else:
                training_set.append(to_concat)
                
        first = True
        for ti in test_index:
            to_concat = dfs[ti].copy()
            if first:
                test_set = [to_concat]
                first = False
            else:
                test_set.append(to_concat)
        
        for t in training_set:
            train_rul = train_rul + t['label'].tolist()
            t['label'] = (t['label'] >= margin).map({True: 1, False: 0})
        if split_size < 1:
            for t in test_set:
                test_rul = test_rul + t['label'].tolist()
                t['label'] = (t['label'] >= margin).map({True: 1, False: 0})
    if split_size < 1:
        return training_set, test_set
    return training_set

## Cost model for threshold optimization and performance evaluation

In [43]:
all_perf = []

In [44]:
BASE_FP = 0.2
BASE_FN = 1

def false_positive_cost(i, is_fault, fault_found):
    return BASE_FP

def false_negative_cost(i, is_fault, fault_found):
    if not fault_found:
        for j in range(1, margin + 1):
            if i + j < is_fault.shape[0] and not is_fault[i + j] or i + j >= is_fault.shape[0]:
                return (margin + 1 - j) * BASE_FN
    else:
        return 0

In [45]:
def threshold_optimization(signal, rul, start, end, n_steps):
    best_cost = sys.maxsize
    best_thr = -1
    all_cost = []
    all_thr = []
    is_fault = (rul == 0)
    
    for thr in np.linspace(start, end, n_steps):
        tmp_cost = 0
        fault_found = False
        for i in range(signal.shape[0]):
            if is_fault[i] and signal[i] >= thr:
                fault_found = True
            if not is_fault[i]:
                fault_found = False
            if not is_fault[i] and signal[i] >= thr:
                tmp_cost += false_positive_cost(i, is_fault, fault_found)
            elif is_fault[i] and signal[i] <= thr:
                tmp_cost += false_negative_cost(i, is_fault, fault_found)
        if tmp_cost < best_cost:
            best_thr = thr
            best_cost = tmp_cost
        all_cost.append(tmp_cost)
        all_thr.append(thr)

    return best_cost, best_thr, all_cost, all_thr

In [46]:
def performance_evaluation(signal, thr, rul):
    fp, fn, tp, tot_p = 0, 0, 0, 0
    cost = 0
    alarm = (signal >= thr)
    anticipation = []
    is_fault = (rul == 0)
    
    fault_found = False
    for i in range(len(rul)):
        if i > 0 and is_fault[i] and not is_fault[i - 1]:
            tot_p += 1
            start = i
        if is_fault[i] and not fault_found and alarm[i]:
            tp += 1
            fault_found = True
            anticipation.append((margin - 1) - (i - start))
        if (i < len(rul) - 1 and is_fault[i] and not is_fault[i + 1] and not fault_found) or (i == len(rul) - 1 and not fault_found):
            fn += 1 
        if is_fault[i] and signal[i] <= thr:
            cost += false_negative_cost(i, is_fault, fault_found)
        if not is_fault[i]:
            fault_found = False
            if alarm[i]:
                fp += 1
                cost += false_positive_cost(i, is_fault, fault_found)
        
    tot_a = sum(anticipation) / 10
    if sum(anticipation) > 0:
        mean_a = mean(anticipation) / 10
    else:
        mean_a = 0
    
    return [cost, mean_a, tp, fn, fp]

In [47]:
def sliding_window_2D(data, w_len, stride=1):
    # Get shifted tables
    m = len(data)
    lt = [data.iloc[i:m-w_len+i+1:stride, :].values for i in range(w_len)]
    # Reshape to add a new axis
    s = lt[0].shape
    for i in range(w_len):
        lt[i] = lt[i].reshape(s[0], 1, s[1])
    # Concatenate
    wdata = np.concatenate(lt, axis=1)
    return wdata


def sliding_window_by_fault(data, cols, w_len, stride=1):
    l_w, l_r = [], []
    cols.pop()
    for gdata in data:
        # Apply a sliding window
        tmp_w = sliding_window_2D(gdata[cols], w_len, stride)
        # Build the RUL vector
        tmp_r = gdata['label'].iloc[w_len-1::stride]
        # Store everything
        l_w.append(tmp_w)
        l_r.append(tmp_r)
    res_w = np.concatenate(l_w)
    res_r = np.concatenate(l_r)
    return res_w, res_r

In [48]:
params_cnn = [{"filters": 1, "kernel_size": 3, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 3, "hidden": [32], "w_len": 5},
              {"filters": 1, "kernel_size": 5, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 5, "hidden": [32], "w_len": 5},
              {"filters": 4, "kernel_size": 5, "hidden": [64, 32], "w_len": 5},
              {"filters": 1, "kernel_size": 3, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 3, "hidden": [32], "w_len": 10},
              {"filters": 1, "kernel_size": 5, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 5, "hidden": [32], "w_len": 10},
              {"filters": 4, "kernel_size": 5, "hidden": [64, 32], "w_len": 10},
              {"filters": 4, "kernel_size": 7, "hidden": [128, 64, 32], "w_len": 10}]

## In loop

In [54]:
with open("experiments/loop1.csv", "w") as file2:
    for seed in seeds:
        for columns in features:
            for params_idx, params in enumerate(params_cnn):

                cnn = keras.models.load_model(models_path + "conv_nn" + "-" + columns + "-s" + str(seed) + "-p" + str(params_idx))
                f = results_path + "conv_nn-" + columns + "-s" + str(seed) + "-p" + str(params_idx)
                with open(f, "rb") as file:
                    training_columns, _, _, best_cost_cnn, best_thr_cnn, _, _, _ = pickle.load(file)

                test_set_cnn = build_dataset_for_ml_model(df, training_columns=training_columns, split_size=1, as_list=True)
                ts_sw, ts_sw_r = sliding_window_by_fault(test_set_cnn, training_columns, params["w_len"])

                while True:
                    start = time()

                    test_preds_cnn = cnn.predict(np.expand_dims(ts_sw[0], axis=0)).ravel()
                    test_signal_cnn =  pd.Series(data=(1 - test_preds_cnn))
                    alarm_signal = (test_signal_cnn >= best_thr_cnn)

                    end = (time() - start)
                    print(f"Executed in {end} seconds")
                    file2.write(f"{start:.3f},{end:.3f}\n")

1/1 [==============================] - 0s 34ms/step
Executed in 0.05147218704223633 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027628183364868164 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02747797966003418 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.029464006423950195 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027246475219726562 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027061939239501953 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.026782989501953125 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027127504348754883 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027475595474243164 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027524709701538086 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027886

1/1 [==============================] - 0s 11ms/step
Executed in 0.028141260147094727 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027488231658935547 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027451276779174805 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027352094650268555 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027214527130126953 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027386188507080078 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.02720046043395996 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.03224325180053711 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.028577804565429688 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02809929847717285 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.0275540

1/1 [==============================] - 0s 11ms/step
Executed in 0.027666807174682617 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027512311935424805 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02750849723815918 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027074098587036133 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027712583541870117 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.032741546630859375 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.028677940368652344 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.028356552124023438 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027620792388916016 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027571916580200195 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02763

1/1 [==============================] - 0s 11ms/step
Executed in 0.02748870849609375 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.026706695556640625 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02735590934753418 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.028035640716552734 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.03280973434448242 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02898406982421875 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.028177976608276367 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.02771782875061035 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027228593826293945 seconds
1/1 [==============================] - 0s 10ms/step
Executed in 0.027185678482055664 seconds
1/1 [==============================] - 0s 11ms/step
Executed in 0.027224779

KeyboardInterrupt: 